In [1]:
import xtgeo
import resqpy
import numpy as np
import pandas as pd
from resqpy.model import Model, new_model
from resqpy.surface import PointSet
from resqpy.crs import Crs

from fmu.sumo.explorer import Explorer
sumo = Explorer("dev")
polygons = sumo.get_polygons_by_uuid("769c204d-9f8f-92ed-ea37-3cef79f282c8")

In [2]:
polydata = polygons.metadata['data']
polydata

{'tagname': 'GL_faultlines_extract_postprocess',
 'is_prediction': True,
 'bbox': {'ymin': 5929956.412841797,
  'zmin': 1601.9921875,
  'xmin': 459423.5546875,
  'ymax': 5937771.147949219,
  'zmax': 1882.09521484375,
  'xmax': 465773.7509765625},
 'format': 'csv',
 'name': 'VOLANTIS GP. Base',
 'stratigraphic': True,
 'vertical_domain': 'depth',
 'is_observation': False,
 'content': 'depth',
 'spec': {'npolys': 6}}

Instantiate new model of polygons

In [3]:
#model = Model(new_epc=True)
model = new_model(epc_file="polygons.epc")

Add "default" coordinate reference system to the model

In [4]:
crs = Crs(model, title="Default Coordinate Reference System")

Add a pointset (different patches of several points) of the polygons data to the model

In [5]:
df = pd.read_csv(polygons.blob)
df

,X,Y,Z,ID
0,460605.935547,5.935551e+06,1668.652466,0
1,460606.017334,5.935521e+06,1666.152466,0
2,460604.712036,5.935503e+06,1664.615845,0
3,460604.477661,5.935432e+06,1662.211304,0
4,460604.581665,5.935408e+06,1661.408447,0
...,...,...,...,...
992,462238.666687,5.933122e+06,1677.452759,5
993,462228.221069,5.933142e+06,1677.865234,5
994,462217.921173,5.933162e+06,1678.282959,5
995,462195.362915,5.933224e+06,1678.972778,5


In [6]:
pointset = PointSet(model, title="Polygons Point Set", crs_uuid=crs.uuid)

# Append whole fmu metadata dict to pointset
pointset.append_extra_metadata(polygons.metadata)

In [7]:
# Add all polygons as different patches
id_string = "POLY_ID" if "POLY_ID" in df.columns else "ID"
for id in range(0, polydata['spec']['npolys']):
    pointset.add_patch(df.loc[df[id_string] == id].to_numpy()[:, :3])

Output our current polygons model

In [8]:
# Write all metadata to epc file
crs.create_xml()
pointset.create_xml()
model.store_epc("polygons.epc")

# Write data to hdf5 file
pointset.write_hdf5()
model.create_hdf5_ext(file_name="polygons.h5")

<Element {http://www.energistics.org/energyml/data/commonv2}EpcExternalPartReference at 0x18cbf222a80>

Try to reopen a new model from the files we just saved

In [9]:
# Recreate the model from epc file
read_model = Model("polygons.epc")
read_model.parts()

['obj_LocalDepth3dCrs_17cbbff3-1a62-11ee-8e6e-00d49ee52a08.xml',
 'obj_PointSetRepresentation_180b0abb-1a62-11ee-833f-00d49ee52a08.xml']

In [10]:
read_model.titles()

['Default Coordinate Reference System', 'Polygons Point Set']

In [11]:
read_model.h5_file_name()

'polygons.h5'

In [12]:
read_model.h5_uuid()

UUID('17ca2e21-1a62-11ee-9f86-00d49ee52a08')

Check that stored array is intact

In [13]:
read_pointset_uuid = read_model.uuid(obj_type="obj_PointSetRepresentation")

read_pointset = PointSet(read_model, uuid=read_pointset_uuid)
print("Shape:", read_pointset.full_array_ref().shape)
print("Sample array piece:", read_pointset.single_patch_array_ref(0)[5])
print("Sample array piece:", read_pointset.single_patch_array_ref(4)[50])

Shape: (997, 3)
Sample array piece: [4.60604711e+05 5.93537787e+06 1.66040405e+03]
Sample array piece: [4.63320068e+05 5.93156164e+06 1.71533508e+03]


In [14]:
print("Shape:", pointset.full_array_ref().shape)
print("Sample array piece:", pointset.single_patch_array_ref(0)[5])
print("Sample array piece:", pointset.single_patch_array_ref(4)[50])

Shape: (997, 3)
Sample array piece: [4.60604711e+05 5.93537787e+06 1.66040405e+03]
Sample array piece: [4.63320068e+05 5.93156164e+06 1.71533508e+03]


Check that extra metadata is intact

In [15]:
read_pointset.extra_metadata

{'masterdata': "{'smda': {'field': [{'identifier': 'DROGON', 'uuid': '00000000-0000-0000-0000-000000000000'}]}}",
 'file': "{'checksum_md5': 'c9f4ebc4ed84efb5a013126e150fe434', 'relative_path': 'realization-2/iter-0/share/results/polygons/basevolantis--gl_faultlines_extract_postprocess.csv'}",
 'access': "{'asset': {'name': 'Drogon'}}",
 'data': "{'tagname': 'GL_faultlines_extract_postprocess', 'is_prediction': True, 'bbox': {'ymin': 5929956.412841797, 'zmin': 1601.9921875, 'xmin': 459423.5546875, 'ymax': 5937771.147949219, 'zmax': 1882.09521484375, 'xmax': 465773.7509765625}, 'format': 'csv', 'name': 'VOLANTIS GP. Base', 'stratigraphic': True, 'vertical_domain': 'depth', 'is_observation': False, 'content': 'depth', 'spec': {'npolys': 6}}",
 'fmu': "{'context': {'stage': 'realization'}, 'iteration': {'name': 'iter-0'}, 'case': {'name': 'rowh_kmd_bleed_fri01', 'user': {'id': 'rowh'}}, 'realization': {'id': 2}}"}

In [16]:
pointset.extra_metadata

{'masterdata': "{'smda': {'field': [{'identifier': 'DROGON', 'uuid': '00000000-0000-0000-0000-000000000000'}]}}",
 'file': "{'checksum_md5': 'c9f4ebc4ed84efb5a013126e150fe434', 'relative_path': 'realization-2/iter-0/share/results/polygons/basevolantis--gl_faultlines_extract_postprocess.csv'}",
 'access': "{'asset': {'name': 'Drogon'}}",
 'data': "{'tagname': 'GL_faultlines_extract_postprocess', 'is_prediction': True, 'bbox': {'ymin': 5929956.412841797, 'zmin': 1601.9921875, 'xmin': 459423.5546875, 'ymax': 5937771.147949219, 'zmax': 1882.09521484375, 'xmax': 465773.7509765625}, 'format': 'csv', 'name': 'VOLANTIS GP. Base', 'stratigraphic': True, 'vertical_domain': 'depth', 'is_observation': False, 'content': 'depth', 'spec': {'npolys': 6}}",
 'fmu': "{'context': {'stage': 'realization'}, 'iteration': {'name': 'iter-0'}, 'case': {'name': 'rowh_kmd_bleed_fri01', 'user': {'id': 'rowh'}}, 'realization': {'id': 2}}"}